In [ ]:
# !pip install pydantic==1.10.13
# !pip install pydantic==2.6.2

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

# Basic Usage

In [4]:
from langchain_zhipu import ChatZhipuAI
from langchain_core.callbacks import BaseCallbackHandler

class CountTokenCallbackHandler(BaseCallbackHandler):
	def on_llm_end(self, response, **kwargs):
		print("-"*20, "Token统计", "-"*20)
		print(response)
		print("-"*20, "--------", "-"*20)
  
llm = ChatZhipuAI(callbacks=[CountTokenCallbackHandler()])


In [3]:
# 或者你想自己临时指定API_KEY
import os
llm = ChatZhipuAI(api_key=os.getenv('ZHIPUAI_API_KEY'))

## invoke

In [5]:
llm.invoke("你是什么模型？")

-------------------- Token统计 --------------------
generations=[[ChatGeneration(text='我是一个人工智能助手模型，基于清华大学 KEG 实验室和智谱AI共同训练的 GLM-130B 模型开发而成。我的任务是针对用户的问题和要求提供适当的答复和支持。', generation_info={'finish_reason': 'stop'}, message=AIMessage(content='我是一个人工智能助手模型，基于清华大学 KEG 实验室和智谱AI共同训练的 GLM-130B 模型开发而成。我的任务是针对用户的问题和要求提供适当的答复和支持。'))]] llm_output={'id': '8554703731291129631', 'created': 1712748744, 'token_usage': {'completion_tokens': 44, 'prompt_tokens': 9, 'total_tokens': 53}, 'model_name': 'glm-4'} run=None
-------------------- -------- --------------------


AIMessage(content='我是一个人工智能助手模型，基于清华大学 KEG 实验室和智谱AI共同训练的 GLM-130B 模型开发而成。我的任务是针对用户的问题和要求提供适当的答复和支持。')

## stream

In [4]:
for s in llm._ask_remote_sse([{"role": "user", "content": "你是什么模型？"}]):
  print(s)

{'id': '8554703868730116790', 'created': 1712748012, 'model': 'glm-4', 'choices': [{'index': 0, 'delta': {'role': 'assistant', 'content': '我'}}]}
{'id': '8554703868730116790', 'created': 1712748012, 'model': 'glm-4', 'choices': [{'index': 0, 'delta': {'role': 'assistant', 'content': '使用的'}}]}
{'id': '8554703868730116790', 'created': 1712748012, 'model': 'glm-4', 'choices': [{'index': 0, 'delta': {'role': 'assistant', 'content': '模型'}}]}
{'id': '8554703868730116790', 'created': 1712748012, 'model': 'glm-4', 'choices': [{'index': 0, 'delta': {'role': 'assistant', 'content': '是'}}]}
{'id': '8554703868730116790', 'created': 1712748012, 'model': 'glm-4', 'choices': [{'index': 0, 'delta': {'role': 'assistant', 'content': '清华大学'}}]}
{'id': '8554703868730116790', 'created': 1712748012, 'model': 'glm-4', 'choices': [{'index': 0, 'delta': {'role': 'assistant', 'content': ' K'}}]}
{'id': '8554703868730116790', 'created': 1712748012, 'model': 'glm-4', 'choices': [{'index': 0, 'delta': {'role': 'as

In [8]:
for s in llm.stream([{"role": "user", "content": "你是什么模型？"}]):
  print(s)

content='我' id='8554703868730090340'
content='使用的' id='8554703868730090340'
content='模型' id='8554703868730090340'
content='是' id='8554703868730090340'
content='清华大学' id='8554703868730090340'
content=' K' id='8554703868730090340'
content='EG' id='8554703868730090340'
content=' 实' id='8554703868730090340'
content='验' id='8554703868730090340'
content='室' id='8554703868730090340'
content='和' id='8554703868730090340'
content='智' id='8554703868730090340'
content='谱' id='8554703868730090340'
content='AI' id='8554703868730090340'
content='共同' id='8554703868730090340'
content='训练' id='8554703868730090340'
content='的' id='8554703868730090340'
content=' GL' id='8554703868730090340'
content='M' id='8554703868730090340'
content=' 模' id='8554703868730090340'
content='型' id='8554703868730090340'
content='，' id='8554703868730090340'
content='一种' id='8554703868730090340'
content='基于' id='8554703868730090340'
content=' Transformer' id='8554703868730090340'
content=' 的' id='8554703868730090340'
content='

In [ ]:
async for s in llm.astream([{"role": "user", "content": "你是什么模型？"}]):
  print(s)

## batch

In [ ]:
llm.batch([
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？"
])

# Tools-Calling

## function

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain.tools import tool

@tool
def search(query: str) -> str:
    """查询 langchan 资料; args: query 类型为字符串，描述用户的问题."""
    return "langchain_chinese 是一个为中国大模型优化的langchain模块"

llm.bind(tools=[convert_to_openai_tool(search)]).invoke("langchain_chinese是啥？请查询本地资料回答。")


## web_search

In [ ]:
from langchain_zhipu import convert_to_web_search_tool
llm.bind(tools=[convert_to_web_search_tool(search_query="周星驰电影")]).invoke("哪部电影好看？")

## retrieval

In [ ]:
from langchain_zhipu import convert_to_retrieval_tool
llm.bind(tools=[convert_to_retrieval_tool(knowledge_id="1772979648448397312")]).invoke("你知道马冬梅住哪里吗？")

In [ ]:
llm.bind(tools=[convert_to_openai_tool(search)])

# 使用官方接口

In [ ]:
from langchain_zhipu import ZhipuAI

client = ZhipuAI()

tools = [
        {
            "type": "function",
            "function": {
                "name": "search",
                "description": "search(query: str) -> str - 查询 langchan 资料; args: query 类型为字符串，描述用户的问题.'",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string"
                        },
                    },
                    "required": ["query"]
                }
            }
        }
    ]

In [ ]:
client.chat.completions.create(
    model="glm-4",
    messages = [
        {
            "role": "user",
            "content": "langchain是什么？请帮我查询"
        }
    ],
    tools = tools,
    tool_choice="auto",
)

In [ ]:
client.chat.completions.create(
    model="glm-4",
    messages = [
        {
            "role": "user",
            "content": "langchain是什么？"
        }
    ],
    tools = tools,
    tool_choice="auto",
)

# 智能体

In [ ]:
!pip install langchainhub

In [ ]:
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain import hub

tools = [search]
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

for chunk in agent_executor.stream({"input": "langchain_chinese是什么？"}):
    print(chunk)


In [ ]:
prompt

# GLM-4V

In [ ]:
from langchain_zhipu import ChatZhipuAI
from langchain_core.prompts import ChatPromptTemplate

llm4v = ChatZhipuAI(model="glm-4v")

prompt = ChatPromptTemplate.from_messages([
    ("human", [
          {
            "type": "text",
            "text": "图里有什么"
          },
          {
            "type": "image_url",
            "image_url": {
                "url" : "https://img1.baidu.com/it/u=1369931113,3388870256&fm=253&app=138&size=w931&n=0&f=JPEG&fmt=auto?sec=1703696400&t=f3028c7a1dca43a080aeb8239f09cc2f"
            }
          }
        ]),
])

(prompt|llm4v).invoke({})

# get_token_ids

In [ ]:
from langchain_zhipu import ChatZhipuAI

llm = ChatZhipuAI(allowed_special=set())
llm.get_token_ids('<|user|>')

In [ ]:
llm.get_num_tokens(text="你好")